In [3]:
import numpy as np
import pandas as pd
import talib 
from talib import abstract
from joblib import delayed,Parallel,cpu_count
cpu_nums=cpu_count()
slice_windows=30

In [4]:
import sys
sys.path.append("../src/")
from utils import *
df = pd.read_csv("../data/raw_data/train.csv")
weight = pd.read_csv("../data/raw_data/asset_details.csv")
Asset_ID = 1

# kfold parameters
cv = 3
plength=0.8
ptrain=0.8
pgap=0.05
cv_method = 'best'
# missing data
timestamp_fill = False
method = 'drop'
weight = weight
df = df
dft = df[df['Asset_ID'] == Asset_ID].sort_values(by=['timestamp'], ascending=True)
# sample
dft = dft.iloc[math.floor(dft.shape[0]/6*5):]
cvs = CVsample(df=dft, ival='timestamp', cv=cv, plength=plength, ptrain=ptrain, pgap=pgap)
cvs.main()

# test
test = dft[dft['timestamp'] > max(cvs.validation_ends)]
tb = talib(test)
test_panel = tb.main()

[train, gap, validation] = cvs.get_df(2)
# feature
tb = talib(train)
train_panel = tb.main()

tb = talib(validation)
val_panel = tb.main()

/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Count']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['amount']=train_data['Count']*train_data['VWAP']
/home/jwangiy/crypto/notebooks/../src/utils.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [5]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.multiprocessing
from joblib import Parallel, delayed
import numpy as np
from tqdm import tqdm

class GruDataset(Dataset):
    """
        Dataset for training
    """
    def __init__(self,dataset, seq_len,train=True):
        #Train denote whether this dataset is used for training(testing otherwise)
        #if train，up and dwon threshold filter will work; if test, no such
        def judge(df,up_threshold=0.2,down_threshold=0.0005):
            #20% abs change in target 
            length=df.shape[0]-seq_len+1
            df=df.copy()
            #float32 to save memory
            df=df.values.astype(np.float32)
            x_list=[]
            y_list=[]
            for idx in tqdm(range(0,length)):
                #像此处的index的部分，取决于
                x=df[idx:idx+seq_len,:-2]
                y=df[idx+seq_len-1,-2]
                #don't study severe price change
                if train:
                    if (abs(y)>down_threshold) and (abs(y)<up_threshold):
                        x_list.append(x)
                        y_list.append(y)
                else:
                        x_list.append(x)
                        y_list.append(y)
            length=len(x_list)
            return [x_list,y_list,length]
        #settle x array and y value in the list so that can get them easily
        #final_set=Parallel(cpu_nums)(delayed(judge)(df) for name,df in dataset.groupby('Asset_ID') ) 
        self.final_set=[judge(df) for name,df in dataset.groupby('Asset_ID')]
        self.length=sum([ self.final_set[i][2] for i in range(len( self.final_set))])
        self.dataset=dataset
        self._x=[ self.final_set[i][0][j] for i in range(len( self.final_set)) for j in range(len( self.final_set[i][0])) ]
        self._y=[ self.final_set[i][1][j] for i in range(len( self.final_set)) for j in range(len( self.final_set[i][1])) ]
    def __len__(self):
        return self.length
    def __getitem__(self, idx):
        x=self._x[idx]
        y=self._y[idx]
        return x,y


In [7]:
#example
test_panel['Asset_ID'] = Asset_ID
gd=GruDataset(test_panel,20)
# print("X: \n\n",gd[100][0],"Y: \n\n",gd[100][1],"\n\n")
# print("length \n\n",len(gd))

100%|█████████████████████████████████████████████████████████████████████████| 25385/25385 [00:00<00:00, 179621.75it/s]


In [8]:

#A temo Dataset, used while cut train and validation  
class TempDataset(Dataset):
        def __init__(self,x,y):
            self.x=x
            self.y=y
        def __len__(self):
            return len(self.x)
        def __getitem__(self, idx):
            return self.x[idx],self.y[idx]

def data_split(df,batch_size=512,train_portion=0.75,seq_len=2400):
    """spilt data in to train set and validation set, and transform 
    into Grudf in model_structure and DataLoader in pytorch"""
    df_train_x=df[:int(len(df)*train_portion)][0]
    df_train_y=df[:int(len(df)*train_portion)][1]
    df_valid_x=df[int(len(df)*train_portion):][0]
    df_valid_y=df[int(len(df)*train_portion):][1]
    df_train=TempDataset(df_train_x,df_train_y)
    df_valid=TempDataset(df_valid_x,df_valid_y)
    #Dataloader
    train_dataloader = DataLoader(df_train, batch_size=batch_size, shuffle=True,drop_last=True)
    valid_dataloader = DataLoader(df_valid, batch_size=batch_size, shuffle=True,drop_last=True)
    return train_dataloader,valid_dataloader


class GRUNN(pl.LightningModule):
    def __init__(self,config ,feature_size:int=52):
        super().__init__()
        self.feature_size=feature_size
        self.lr=config['lr']
        self.l2=config['l2']
        self.dropout=config['dropout']
        self.layer_norm = nn.LayerNorm(feature_size)
        self.gru = nn.GRU(feature_size,int(feature_size*2/3), 2, batch_first=True, dropout=self.dropout)#hiddenlayer/inputlayer=2/3
        self.linear = nn.Linear(int(feature_size*2/3), 1) #dense

    def forward(self, x):
        #x = self.layer_norm(x)#actually this step is not necessary
        out, _ = self.gru(x)#return output,h_n
        out = self.linear(out[:, -1])
        return out

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr,weight_decay=self.l2)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_pred = self(x)
        # weight = torch.Tensor([1., 0.072, 1.]).cuda()
        # loss = F.cross_entropy(y_pred, y, weight=weight)
        loss_fun = nn.MSELoss()
        loss = loss_fun(y_pred, y)
        # loss = F.binary_cross_entropy_with_logits(y_pred, y.view(-1, 1).to(torch.float))
        self.log('train/loss', loss.item())
        return loss

    def predict_step(self, batch, batch_idx: int , dataloader_idx: int = None):
        x,y=batch
        return [self(x),y]

In [11]:
config = {
    "lr":1e-4,
    "l2":0.2,
    "dropout":0.2,
}
# Transfer to accelerator
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:3" if use_cuda else "cpu")

torch.manual_seed(0)
torch.cuda.manual_seed(0)
np.random.seed(0)

model = GRUNN(config
).to(device)

In [15]:
training_losses = []
validation_losses = []
min_validation_loss = np.Inf
validate_every = 2
EPOCHS = 5

# Set to train mode
model.train()

for epoch in tqdm(range(EPOCHS)):
    # Initialize hidden and cell states with dimension:
    # (num_layers * num_directions, batch, hidden_size)
#     states = model.init_hidden_states(BATCH_SIZE)
    running_training_loss = 0.0

    # Begin training
    for idx, (x_batch, y_batch) in enumerate(training_dl):
        # Convert to Tensors
        x_batch = x_batch.float().to(device)
        y_batch = y_batch.float().to(device)

        # Truncated Backpropagation
#         states = [state.detach() for state in states]

        optimizer.zero_grad()

        # Make prediction
        output, states = model(x_batch, states)

        # Calculate loss
        loss = criterion(output[:, -1, :], y_batch.squeeze())
        loss.backward()
        running_training_loss += loss.item()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

    # Average loss across timesteps
    training_losses.append(running_training_loss / len(training_dl))

    if epoch % validate_every == 0:
        # Set to eval mode
        model.eval()

        validation_states = model.init_hidden_states(BATCH_SIZE)
        running_validation_loss = 0.0

        for idx, (x_batch, y_batch) in enumerate(validation_dl):
            # Convert to Tensors
            x_batch = x_batch.float().to(device)
            y_batch = y_batch.float().to(device)

            validation_states = [state.detach() for state in validation_states]
            output, validation_states = model(x_batch, validation_states)
            validation_loss = criterion(output[:, -1, :], y_batch.squeeze())
            running_validation_loss += validation_loss.item()

        validation_losses.append(running_validation_loss / len(validation_dl))
        # Reset to training mode
        model.train()

        is_best = running_validation_loss / len(validation_dl) < min_validation_loss

        if is_best:
            min_validation_loss = running_validation_loss / len(validation_dl)
            save_checkpoint(epoch + 1, min_validation_loss, model.state_dict(), optimizer.state_dict())

# Visualize loss
epoch_count = range(1, len(training_losses) + 1)
plt.plot(epoch_count, training_losses, 'r--')
plt.legend(['Training Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

val_epoch_count = range(1, len(validation_losses) + 1)
plt.plot(val_epoch_count, validation_losses, 'b--')
plt.legend(['Validation loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

  0%|                                                                                             | 0/5 [00:00<?, ?it/s]


NameError: name 'training_dl' is not defined